In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objs as go

from p2x2p.utils.utils import get_config

# Get the configuration
config = get_config()

# Use autoreload magic so that .py files can modified without having to restart the kernel
%load_ext autoreload
%autoreload 2

### TTF price information
Weekly TTF prices for natural gas can be obtained from: https://tradingeconomics.com/commodity/eu-natural-gas

You can only download the data in svg fomrat for free, but the data trace can be extracted from the svg file by simply copying the data trace from the svg file and scaling it properly by following the instructions below. 
1. Select 1 week as the time interval to avid getting unnessaryly many data points.
2. Select the interval you are interested in, e.g., 2015-12-22 to 2024-01-18.
3. Check the actual start and end date in the browser and modify the code below accordingly.
4. Check the first and maximal price and modify the code below accordingly.
5. Download the data in svg format.
6. Open the svg file in an [svg viewer](https://www.svgviewer.dev/) and identify the svg path containing the data, looks like: `<path fill="none" d="M ..."`
7. Copy the data (the part within quotation marks after `d=`) into the cell below

In [ ]:
start_date = "2015-12-28 00:00:00"
end_date = "2024-01-15 00:00:00"
first_price = 14.79
max_price = 339.20
svg_data = "ADD SVG PATH DATA HERE"

In [ ]:
# Extract raw price and date values in pixel coordinates
split_data = svg_data.split(" ")
price_raw = np.array(split_data[2::3], dtype=np.float64)
date_number = np.array(split_data[1::3], dtype=np.float64)

In [ ]:
ttf_data = pd.DataFrame()

# Interpolate the relative datetimes in pixels to absolute datetimes rounded to the nearest hour
relative_time = date_number.copy()
relative_time = relative_time / relative_time.max()  # Normalize to [0, 1]
# Known true datetimes for the first and last relative times
start_datetime = pd.Timestamp(start_date)
end_datetime = pd.Timestamp(end_date)
# Calculate total duration in seconds between the start and end datetimes
total_duration = (end_datetime - start_datetime).total_seconds()

ttf_data['date'] = start_datetime + pd.to_timedelta(relative_time * total_duration, unit='s')
ttf_data['date'] = ttf_data['date'].dt.round('h')

# Get the linear equation mapping coordinates in pixels to price
price_diff_raw = price_raw[0] - price_raw.min()
price_diff = first_price - max_price
slope = price_diff / price_diff_raw
bias = first_price - slope*price_raw[0]

# Convert the raw price values into Euros/MWh
ttf_data['TTF (Euro/MWh)'] = price_raw*slope + bias

In [ ]:
# Create an hourly time index from start to end and interpoalte missing prices
hourly_time_index = pd.date_range(start=start_datetime, end=end_datetime, freq='h')
ttf_data = ttf_data.set_index('date').reindex(hourly_time_index)
ttf_data['TTF (Euro/MWh)'] = ttf_data['TTF (Euro/MWh)'].interpolate(method='linear')
ttf_data.reset_index(inplace=True)
ttf_data.rename(columns={'index': 'date'}, inplace=True)
ttf_data['date'] = (ttf_data['date'].dt.tz_localize('UTC').dt.tz_convert('Europe/Helsinki'))
ttf_data

In [ ]:
# Visualize the data to validate that the conversion was succesful
fig = go.Figure()
# Add data traces
fig.add_trace(go.Scattergl(x=ttf_data['date'], y=ttf_data['TTF (Euro/MWh)'], mode='lines', line=dict(color='black'), name='TTF'))
# Show the plot
fig.update_layout(title="Weekly TTF prices"),
fig.update_xaxes(title="Date"),
fig.update_yaxes(title="TTF price (€/MWh)"),
fig.show()

In [ ]:
# Create a dataframe and save the data to disk
ttf_data.to_csv(config['data_paths']['ttf_natural_gas'], index=False)
ttf_data.head()